#### Title: 00_preprocess_MIMICIII.ipynb

#### Description: Create dataset for predicting Glasgow Coma Scale based on MIMIC-III database 

#### Author: Fabian Kahl

In [ ]:
import pandas as pd
import numpy as np

# Source: https://physionet.org/content/mimiciii/1.4/
data_path = '../data/physionet.org/files/mimiciii/1.4/'

# Function to merge GCS (Glasgow Coma Scale) scores with backward tolerance for matching chart events
def get_values_backward(df_gcs, items_ids, df, df_ids, tolerance = pd.Timedelta('24h')):
    df_out = df_gcs[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score']].copy()
    df.sort_values(['CHARTTIME'], inplace=True)
    df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME'])
    df = df.dropna(subset=['CHARTTIME'])
    for i in items_ids:
        df_i = df[df['ITEMID'] == i]
        item_name = df_ids[df_ids['ITEMID'] == i]['LABEL'].iloc[0]
        df_i = df_i.rename(columns={'VALUE': item_name})
        df_i = df_i[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', item_name]]
        df_out = pd.merge_asof(df_out, df_i,
                                on='CHARTTIME',
                                by=['SUBJECT_ID', 'HADM_ID'],
                                direction='backward', tolerance=tolerance)
    return df_out

# Function to merge GCS scores with corresponding item values without time-based tolerance
def get_values(df_gcs, items_ids, df, df_ids):
    df_out = df_gcs[['SUBJECT_ID', 'HADM_ID', 'gcs_score']].copy()
    df.sort_values(['SUBJECT_ID', 'HADM_ID'], inplace=True)
    df_out.sort_values(['SUBJECT_ID', 'HADM_ID'], inplace=True)
    for i in items_ids:
        df_i = df[df['ITEMID'] == i]
        item_name = df_ids[df_ids['ITEMID'] == i]['LABEL'].iloc[0]
        df_i = df_i.rename(columns={'VALUE': item_name})
        df_i = df_i[['SUBJECT_ID', 'HADM_ID', item_name]]
        df_out = df_out.merge(df_i, on=['SUBJECT_ID', 'HADM_ID'], how='left')
    return df_out
    
# Load data, partially in chunks due to large size of files
chunk_size = 10**6
chartevents_chunks = pd.read_csv(data_path+'CHARTEVENTS.csv.gz', low_memory=False, chunksize=chunk_size)
d_items = pd.read_csv(data_path+'D_ITEMS.csv.gz', low_memory=False)
icustays = pd.read_csv(data_path+'ICUSTAYS.csv.gz', low_memory=False)
labevents = pd.read_csv(data_path+'LABEVENTS.csv.gz', low_memory=False)
d_labitems = pd.read_csv(data_path+'D_LABITEMS.csv.gz', low_memory=False)
inputevents = pd.read_csv(data_path+'INPUTEVENTS_CV.csv.gz', low_memory=False)
prescriptions = pd.read_csv(data_path+'PRESCRIPTIONS.csv.gz', low_memory=False)
diagnoses_icd = pd.read_csv(data_path+'DIAGNOSES_ICD.csv.gz', low_memory=False)
d_icd_diagnoses = pd.read_csv(data_path+'D_ICD_DIAGNOSES.csv.gz', low_memory=False)

# Heart Rate, Arterial BP Mean, Non Invasive Blood Pressure systolic, SpO2, Respiratory Rate, Temperature C
vital_signs_ids = [211, 220045, 51, 52, 220179, 646, 220277, 618, 619, 224690, 676, 223762]

# Identify GCS-related item IDs
gcs_items = d_items[d_items['LABEL'].str.contains('GCS Total', case=False, na=False)]
gcs_item_ids = gcs_items['ITEMID'].tolist()

# Filter for GCS events and vital signs events
gcs_events_list = []
vital_signs_list = []
i = 0
for chunk in chartevents_chunks:
    i+=1
    # Filter the chunk for GCS-related events
    gcs_chunk = chunk[chunk['ITEMID'].isin(gcs_item_ids)]
    vital_signs_chunk = chunk[chunk['ITEMID'].isin(vital_signs_ids)]
    gcs_events_list.append(gcs_chunk)
    vital_signs_list.append(vital_signs_chunk)

# Concatenate the filtered GCS and vital sign events into full dataframes
gcs_events = pd.concat(gcs_events_list, ignore_index=True)
vital_signs = pd.concat(vital_signs_list, ignore_index=True)

# Define lab item IDs for Glucose, Sodium, Potassium, Bicarbonate, pH
lab_items_ids = [50809, 50824, 50822, 50882, 50820]

# Heart Rate, Arterial BP Mean, Non Invasive Blood Pressure systolic, SpO2, Respiratory Rate, Temperature C
# Without duplicates
vital_signs_ids = [220045, 52, 220179, 646, 618, 676]
# Heart Rate, Arterial BP Mean, Non Invasive Blood Pressure systolic, SpO2, Respiratory Rate, Temperature C

# Clean the dataframes by removing and correcting erroneous data
labevents.dropna(subset=['HADM_ID'], inplace=True)
labevents['HADM_ID'] = labevents['HADM_ID'].astype(int)
gcs_events['CHARTTIME'] = pd.to_datetime(gcs_events['CHARTTIME'])
gcs_events = gcs_events.rename(columns={'VALUE': 'gcs_score'})
gcs_events.dropna(subset=['gcs_score'], inplace=True)
gcs_events.drop_duplicates(subset=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score'], keep='first', inplace=True)
gcs_events['gcs_score'] = gcs_events['gcs_score'].astype(int)
gcs_events.sort_values(['CHARTTIME'], inplace=True)
vital_signs_adjusted = vital_signs.copy()
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(211, 220045) # Heart Rate --> Heart Rate
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(51, 52) # Arterial BP [Systolic] --> Arterial BP Mean
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(619, 618) # Respiratory Rate Set --> Respiratory Rate
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(224690, 618) # Respiratory Rate (Total) --> Respiratory Rate
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(223762, 676) # Temperature Celsius --> Temperature C
vital_signs_adjusted['ITEMID'] = vital_signs_adjusted['ITEMID'].replace(220277, 646) # O2 saturation pulseoxymetry --> SpO2
labevents.replace('GREATER THAN 999', 1000, inplace=True)
labevents.replace('>500', 501, inplace=True)
labevents.replace('<10', 9, inplace=True)
labevents.replace('GREATER THAN 500', 501, inplace=True)
labevents.replace('ERROR', np.nan, inplace=True)
labevents.replace('INTERFERING SUBSTANCES', np.nan, inplace=True)
labevents.replace('DISREGARD PREVIOUS RESULT OF 88', 88, inplace=True)
labevents.replace('ERROR, DISREGARD PREVIOUS RESULT OF 131', 131, inplace=True)
labevents.replace('LESS THAN 10', 9, inplace=True)
labevents.replace('UNABLE TO MEASURE POTASSIUM, OUT OF REPORTABLE RANGE', np.nan, inplace=True)
labevents.replace('GREATER THAN 10', 11, inplace=True)
labevents.replace('DISREGARD PREVIOUS RESULT OF 3.5', 3.5, inplace=True)
labevents.replace('ERROR, DISREGARD PREVIOUS RESULT OF 4.6', 4.6, inplace=True)
labevents.replace('GREATER THAN 45', 46, inplace=True)
labevents.replace('GREATER THAN 40', 41, inplace=True)
labevents.replace('LESS THAN 5', 4, inplace=True)
labevents.replace('<5', 4, inplace=True)
labevents.replace('GREATER THAN 50', 51, inplace=True)
labevents.replace('LESS THAN 5.0', 4, inplace=True)
labevents.replace('>50', 51, inplace=True)
labevents.replace('VERIFIED BY REPLICATE ANALYSIS', np.nan, inplace=True)
labevents.replace('<5.0', 4, inplace=True)
labevents.replace('VERIFIED BY REPEAT ANALYSIS', np.nan, inplace=True)
labevents.replace('DISREGARD PREVIOUS RESULT 24,SPECIMEN MISLABELLED', 24, inplace=True)
labevents.replace('7.45 PLEURAL FLUID', 7.45, inplace=True)
labevents.replace('DISREGARD', np.nan, inplace=True)
labevents.replace('DISREGARD PREVIOUSLY REPORTED RESULT OF 7.29', 7.29, inplace=True)
labevents.replace('QUANTITY NOT SUFFICIENT', np.nan, inplace=True)
labevents.replace('DISREGARD RESULTS', np.nan, inplace=True)
labevents.replace('DISREGARD PREVIOUS RESULT OF 7.44', 7.44, inplace=True)
labevents.replace('GREATER THAN 7.55', 7.56, inplace=True)
labevents.dropna(subset=['VALUE'], inplace=True)

# Filter for sedative drugs and create a feature indicating sedative use
sedative_drugs = ['Midazolam', 'Propofol', 'Morphine', 'Fentanyl']
sedative = prescriptions[['SUBJECT_ID', 'HADM_ID', 'ENDDATE', 'DRUG']].copy()
sedative['VALUE'] = sedative['DRUG'].isin(sedative_drugs).astype(int)
sedative['ITEMID'] = 'is_sedative'
sedative = sedative[sedative['VALUE'] == 1].drop(['DRUG'], axis=1)
sedative = sedative.rename(columns={'ENDDATE': 'CHARTTIME'})

# Extract diagnosis information for specific neurological conditions
diagnosis_substrings = ['intracranial hemorrhage', 'encephalitis', 'concussion', 'meningitis', 'epilepsy', 'hydrocephalus']
diagnoses = None
for i in diagnosis_substrings:
    relevant_diagnoses = d_icd_diagnoses[d_icd_diagnoses['LONG_TITLE'].str.contains(i, case=False)]
    diagnosis_codes = list(relevant_diagnoses['ICD9_CODE'])
    diagnosis = diagnoses_icd[diagnoses_icd['ICD9_CODE'].isin(diagnosis_codes)].copy()
    diagnosis['ITEMID'] = i
    diagnosis['VALUE'] = 1
    if diagnoses is None:
        diagnoses = diagnosis
    else:
        diagnoses = pd.concat([diagnoses, diagnosis], ignore_index=True)
diagnoses.drop(['ROW_ID', 'SEQ_NUM', 'ICD9_CODE'], axis=1, inplace=True)
diagnoses.drop_duplicates(inplace=True)

# Extract lab test features
lab_tests_features = get_values_backward(gcs_events, lab_items_ids, labevents, d_labitems)
lab_tests_features['Glucose'] = lab_tests_features['Glucose'].astype(float)
lab_tests_features['Sodium, Whole Blood'] = lab_tests_features['Sodium, Whole Blood'].astype(float)
lab_tests_features['Potassium, Whole Blood'] = lab_tests_features['Potassium, Whole Blood'].astype(float)
lab_tests_features['Bicarbonate'] = lab_tests_features['Bicarbonate'].astype(float)
lab_tests_features['pH'] = lab_tests_features['pH'].astype(float)

# Extract vital signs features
vital_signs_features = get_values_backward(gcs_events, vital_signs_ids, vital_signs_adjusted, d_items)
vital_signs_features['Heart Rate'] = vital_signs_features['Heart Rate'].astype(float)
vital_signs_features['Arterial BP Mean'] = vital_signs_features['Arterial BP Mean'].astype(float)
vital_signs_features['Non Invasive Blood Pressure systolic'] = vital_signs_features['Non Invasive Blood Pressure systolic'].astype(float)
vital_signs_features['SpO2'] = vital_signs_features['SpO2'].astype(float)
vital_signs_features['Respiratory Rate'] = vital_signs_features['Respiratory Rate'].astype(float)
vital_signs_features['Temperature C'] = vital_signs_features['Temperature C'].astype(float)

# Extract sedative features
dummy = pd.DataFrame(columns=['ITEMID', 'LABEL'], data=[['is_sedative','is_sedative']])
sedative_features = get_values_backward(gcs_events, ['is_sedative'], sedative, dummy)

# Extract diagnosis features
dummy = pd.DataFrame({'ITEMID': diagnosis_substrings,
                     'LABEL': diagnosis_substrings})
diagnosis_features = get_values(gcs_events, diagnosis_substrings, diagnoses, dummy)

# Merge all features to one dataset
dataset = gcs_events[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score']]
dataset = pd.merge(dataset, lab_tests_features, on=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score'], how='left')
dataset = pd.merge(dataset, vital_signs_features, on=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score'], how='left')
dataset = pd.merge(dataset, sedative_features, on=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'gcs_score'], how='left')

# Shuffle the dataset to ensure random selection of duplicates
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Drop duplicates based on 'SUBJECT_ID', keeping the first occurrence after shuffling
dataset.drop_duplicates(subset=['SUBJECT_ID'], keep='first', inplace=True)
dataset.columns = dataset.columns.str.replace(r'[^\w\s]', '', regex=True)

dataset.to_csv(data_path+'dataset.csv', index=False)